# Stage 1 - Exploratory Data Analysis

---

Kelompok 2:

*   Oei, Gabrielle Maureen B.S
*   Ummi Sarah
*   Arini Arumsari
*   Althaaf Athaayaa Daffa Qushayyizidane
*   Ruslan Hadi
*   Muhammad Ikhsan Hendriyana
*   Taufik Ismail
*   Jackson
*   Tsany Sutia

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE


print('numpy version : ',np.__version__)
print('pandas version : ',pd.__version__)
print('seaborn version : ',sns.__version__)

In [ ]:
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.set_style("whitegrid")
sns.color_palette("dark")
plt.style.use("fivethirtyeight")

# **Load and Descriptive Statistics**

## Load Dataset

In [ ]:
# Melakukan import csv
df = pd.read_csv("train.csv")
df.sample(5)

## Descriptive Statictics

In [ ]:
# Mengecek apakah ada data yang NaN/Null
print(df.isnull().values.any())
print(df.isna().sum())

In [ ]:
# Mengecek Datatype tiap kolom berserta non-null kolom
df.info()

In [ ]:
# Pengelompokan kolom berdasarkan jenisnya
num = ['int64', 'int32', 'int16', 'float64', 'float32', 'float16']
num = df.select_dtypes(include=num)
num.drop(columns=['id'],inplace=True)
num = num.columns
cats = ['Gender','Vehicle_Age','Vehicle_Damage']

In [ ]:
df[num].describe()

In [ ]:
df[cats].describe()

In [ ]:
# Mengecek unique value, modus, dan frekuensi modus dari tiap kolom
for i in df[num]:
  x = df[num][i].mode()[0]
  y = df[num][i].value_counts()[x]
  z = df[num][i].nunique()
  print(f'Terdapat {z} unique values dan modus dari kolom {i} = {x} dengan frekuensi sebesar {y}')

In [ ]:
# Mengecek ukuran penyebaran

for i in df[num]:
  minv = df[num][i].min()
  maxv = df[num][i].max()
  Q1 = df[num][i].quantile(.25)
  Q3 = df[num][i].quantile(.75)
  range = maxv-minv
  var = round(df[num][i].var(),2)
  iqr = Q3-Q1
  print (f'Ukuran penyebaran dari kolom {i}, variance = {var}, range = {range}, IQR = {iqr}')

## Kesimpulan

1. Apakah ada kolom dengan tipe data kurang sesuai, atau nama kolom dan isinya
kurang sesuai?
    > Dari 12 kolom, terdapat beberapa kolom yang perlu disesuaikan baik dari tipe data maupun nama kolomnya agar lebih mudah dimengerti,
    > * id = ID unik untuk tiap customer. **Sudah sesuai**.
    > * Gender = Jenis kelamin dari customer. **Sudah sesuai**.
    > * Age = Usia customer. **Sudah sesuai**.
    > * Driving_License = 0 : customer tidak memiliki izin mengemudi, 1 : customer memiliki izin mengemudi. **Sudah sesuai**.
    > * Region_Code = Kode unik untuk tiap wilayah customer. **Sudah sesuai**.
    > * Previously_Insured = 0 : Customer belum memiliki 'Asuransi Kendaraan', 1 : Customer sudah memiliki 'Asuransi Kendaraan'. **Sudah sesuai**.
    > * Vehicle_Age = Usia dari kendaraan milik customer. **Lebih baik jika diubah menjadi integer, contoh dengan melakukan *label encoding***.
    > * Vehicle_Damage = 0 : Kendaraan customer belum pernah rusak, 1: Kendaraan customer sudah pernah rusak. **Tipe data lebih baik diseragamkan dengan boolean yang lain yakni integer**.
    > * Annual_Premium = Premi tahunan yang harus dibayar oleh customer. **Sudah sesuai**.
    > * Policy_Sales_Channel = Kode channel/media yang digunakan untuk menghubungi customer. **Sudah sesuai**.
    > * Vintage = Jumlah hari customer sudah bergabung dengan perusahaan. **Lebih baik diubah menjadi Join_Duration**. 
    > * Response = 0 : Customer tidak tertarik 'Asuransi Kendaraan', 0 : Customer tertarik 'Asuransi Kendaraan'**Sudah sesuai**.

2. Apakah ada kolom yang memiliki nilai kosong? Jika ada, apa saja?
    > Dari 12 kolom tidak ditemukan nilai kosong.

3. Apakah ada kolom yang memiliki nilai summary agak aneh?
(min/mean/median/max/unique/top/freq)
    > Dari kolom ***Age***, ***Previously_Insured***, median < mean sehingga ada berkemungkinan persebaran data *positive skewed*, sedangkan kolom ***Policy_Sales_Channel***, ***Region_Code*** median > mean, yang kemungkinan *negative skewed*.


# **Univariate Analysis**

In [ ]:
# Pengelompokan kolom berdasarkan jenisnya
print(num)
print(cats)

In [ ]:
for i in range(0, len(num)):
    plt.subplot(1, len(num), i+1)
    sns.boxplot(y=df[num[i]], color = '#E7CBA9', data=df, orient = 'v')
    plt.tight_layout()

#### Hasil Observasi = 
* Age : Distribusi data tidak normal. Umur pelanggan bervarian dari 20 tahun hingga 85 tahun, dengan kebanyakan pelanggan berusia antara 24 - 49 tahun. 
* Driving_License : Merupakan boolean. Data tidak terdistribusi dengan normal. Kebanyakan pelanggan terkategori 1 (Memiliki izin mengemudi).
* Region_Code : Distribusi data normal. Dengan median di kode wilayah 28.
* Previously_Insured : Merupakan boolean. Data terdistribusi dengan rata antara 1 dan 0.
* Annual_Premium : Data terdistribusi normal, dengan kebanyakan annual premium sekitar 0 - 50000 (dibawah 100000), Terdapat banyak outliers yang nominalnya hingga diatas 400000.
* Policy_Sales_Channel : Distribusi data tidak normal dengan nilai tengah berada di nilai ± 130.
* Vintage : Distribusi data normal, dengan median di nilai ± 151.
* Response : Merupakan boolean. Data tidak terdistribusi dengan normal. Kebanyakan pelanggan terkategori 0 (Tidak berminat).

In [ ]:
for i in range(0, len(num)):
    plt.subplot(2, len(num), i+1)
    sns.distplot(df[num[i]], color='#E7CBA9')
    plt.tight_layout(pad=2)

#### Hasil Observasi = 
* Age : Persebaran *positive skewed*.
* Driving_License : Merupakan bimodal, persebaran tidak rata, mayoritas nilai 1.
* Region_Code : Persebaran data normal.
* Previously_Insured : Merupakan bimodal, persebaran data rata antara 1 dan 0, dimana 0 terlihat sedikit lebih banyak.
* Annual_Premium : Distribusi data mendekati normal namun terdapat outliers diatas batas atas.
* Policy_Sales_Channel : Distribusi data tidak normal, cenderung *negative skewed*.
* Vintage : Persebaran data normal.
* Response : Merupakan bimodal, persebaran tidak rata, mayoritas nilai 0.

In [ ]:
for i in range(0, len(num)):
    plt.subplot(1, len(num), i+1)
    sns.violinplot(y=df[num[i]], color='#E7CBA9', orient='v')
    plt.tight_layout()

#### Hasil Observasi = 
* Age : Mayoritas nilai 24.
* Driving_License : Mayoritas nilai 1.
* Region_Code : Mayoritas nilai 28.
* Previously_Insured : Terdapat 2 nilai 1 dan 0, terlihat dari chart nilai 0 sedikit lebih banyak.
* Annual_Premium : Kebanyakan nilai < 30000.
* Policy_Sales_Channel : Mayoritas nilai ±151.
* Vintage : Data merata ditiap nilainya.
* Response : Mayoritas nilai 0.

In [ ]:
for i in range(0, len(cats)):
    plt.subplot(len(cats), 1, i+1)
    sns.countplot(y=df[cats[i]], color='#E7CBA9')
    plt.tight_layout()


#### Hasil Observasi = 
* Gender : Mayoritas nilai **'Male'**.
* Vehicle_Age : Mayoritas nilai **'1-2 Year'**, disusul dengan '<1 Year' dan yang paling sedikit '>2 Years'.
* Vehicle_Damage : Mayoritas nilai **'Yes'**.

## Kesimpulan

Yang harus di-follow up saat data pre-processing?

> *   Outliers
> *   Distribusi data yang tidak seimbang
> *   Class Imbalance


# **Multivariate Analysis**

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(df[num].corr(), cmap='Purples', annot=True, fmt='.2f')

In [ ]:
#plt.figure(figsize=(15, 15))
#sns.pairplot(df, diag_kind='kde',hue='Response')

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(num)): 
    ax = fig.add_subplot(3, 3, i+1) 
    sns.stripplot(ax=ax, data=df, x='Gender', y=num[i]) 
    plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(num)): 
    ax = fig.add_subplot(3, 3, i+1) 
    sns.stripplot(ax=ax, data=df, x='Vechile_Damage', y=num[i]) 
    plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(num)): # untuk setiap kolom numerik
    ax = fig.add_subplot(3, 3, i+1) # kita set posisi catplot/stripplotnya di layout
    sns.stripplot(ax=ax, data=df, x='Vechile_Age', y=num[i]) # gambar catplot/stripplotnya
    plt.tight_layout()

## Kesimpulan
1. Bagaimana korelasi antara masing-masing feature dan label. Kira-kira feature mana saja yang paling relevan dan harus dipertahankan?
>



2. Bagaimana korelasi antar-feature, apakah ada pola yang menarik?
>

# **Business Insight**

## Kesimpulan